In [ ]:
# Jinko specifics imports & initialization
# Please fold this section and do not edit it
import jinko_helpers as jinko
import sys

sys.path.insert(0, "../lib")

# Connect to Jinko (see README.md for more options)
jinko.initialize()

In [ ]:
# Cookbook specifics imports
import json
import io
import zipfile
import pandas as pd

## Select calibration of interest

In [ ]:
"""
An optional folder location to store debug files
"""

folder = "."
# Set this to True to write debugging files
write_files = True

""" 
Calibration short id can be retrieved from the URL of your calibration in Jinko, pattern is `https://jinko.ai/<calibration_short_id>`
"""
calibration_short_id = "ca-2MzC-vXzw"
# Choose a spefic revision. By default we return the last version
revision = 5
# Choose a specific label. By default we return the last version
label = None
response = jinko.get_project_item(
    sid=calibration_short_id, revision=revision, label=label
)
calibration_core_item_id, calibration_snapshot_id = (
    response["coreId"]["id"],
    response["coreId"]["snapshotId"],
)

# # Uncomment the following if you want to use the latest completed or stopped version
# response = jinko.get_latest_calib_with_status(shortId=calibration_short_id, statuses=["completed", "stopped"])
# calibration_core_item_id, calibration_snapshot_id = response["coreItemId"], response["snapshotId"]

print(
    f"Picked Calibration with coreItemId: {calibration_core_item_id}, snapshotId: {calibration_snapshot_id}"
)

status = jinko.make_request(
    f"/core/v2/calibration_manager/calibration/{calibration_core_item_id}/snapshots/{calibration_snapshot_id}/status",
    method="GET",
).text
print(f"It has status: {status}")

## Debug data tables

In [ ]:
sorted_patients = jinko.make_request(
  path=f"/core/v2/result_manager/calibration/{calibration_core_item_id}/snapshots/{calibration_snapshot_id}/sorted_patients",
  method="POST",
  json={"sortBy": "optimizationWeightedScore", 
        "iteration": 10}
).json()
if len(sorted_patients) > 0:
    best_patient = sorted_patients[0]
    print(f"best_patient: {best_patient}")

    response = jinko.make_request(
      path=f"/core/v2/result_manager/calibration/{calibration_core_item_id}/snapshots/{calibration_snapshot_id}/augment_data_tables",
      method="POST",
      json={"patientId": best_patient["patientNumber"], 
            "iteration": best_patient["iteration"]}
    )
    archive = zipfile.ZipFile(io.BytesIO(response.content))
    filenames = archive.namelist()
    print(f'CSV files in archive: {filenames}')
    for filename in filenames:
      print(f'Extracting : {filename}')
    csv_content = archive.read(filename).decode('utf-8')
    datatable = pd.read_csv(io.StringIO(csv_content))
    display(datatable)